In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import re
import tensorflow_datasets as tfds

In [23]:
raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name = "imdb_reviews",
    split = ["train[:90%]", "train[90%:]", "test"],
    as_supervised = True
)

In [24]:
train_set = raw_train_set.shuffle(5000, seed = 42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [25]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8"))
    print("label", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
label 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development 

In [26]:
vocab_size = 1000

In [27]:
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens = vocab_size)

In [28]:
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

In [29]:
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, 128, mask_zero = True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [30]:
model.compile(loss = "binary_crossentropy", optimizer = "nadam",
              metrics = ["accuracy"])

In [32]:
history = model.fit(train_set, validation_data = valid_set, epochs = 5)

Epoch 1/5
704/704 [==============================] - 24s 34ms/step - loss: 0.3974 - accuracy: 0.8203 - val_loss: 0.3283 - val_accuracy: 0.8540
Epoch 2/5
704/704 [==============================] - 23s 33ms/step - loss: 0.3206 - accuracy: 0.8629 - val_loss: 0.3115 - val_accuracy: 0.8712
Epoch 3/5
704/704 [==============================] - 24s 34ms/step - loss: 0.3010 - accuracy: 0.8719 - val_loss: 0.2984 - val_accuracy: 0.8732
Epoch 4/5
704/704 [==============================] - 24s 34ms/step - loss: 0.2804 - accuracy: 0.8825 - val_loss: 0.2999 - val_accuracy: 0.8724
Epoch 5/5
704/704 [==============================] - 23s 33ms/step - loss: 0.2612 - accuracy: 0.8927 - val_loss: 0.3021 - val_accuracy: 0.8728


In [33]:
model.evaluate(test_set)

782/782 [==============================] - 11s 14ms/step - loss: 0.3123 - accuracy: 0.8673


[0.31229883432388306, 0.8673200011253357]

In [34]:
model.predict(test_set.take(1))

array([[0.90012205],
       [0.97965014],
       [0.07511107],
       [0.35713652],
       [0.99318564],
       [0.97297794],
       [0.99601656],
       [0.99159616],
       [0.48128152],
       [0.81453866],
       [0.02436734],
       [0.85723984],
       [0.81507677],
       [0.16622739],
       [0.28210062],
       [0.46126086],
       [0.971219  ],
       [0.3428719 ],
       [0.8949495 ],
       [0.00440716],
       [0.00939815],
       [0.9640079 ],
       [0.0678355 ],
       [0.26761574],
       [0.9824569 ],
       [0.5051686 ],
       [0.19485536],
       [0.29720235],
       [0.41259533],
       [0.7853859 ],
       [0.91356295],
       [0.9719829 ]], dtype=float32)

In [35]:
for review, label in test_set.take(1):
    print(label)

tf.Tensor([1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 1 1 1], shape=(32,), dtype=int64)
